In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [67]:
team_stats = pd.read_csv('../data/FS_champ_team_2021.csv')
games = pd.read_csv('../data/FS_champ_games_2021.csv')
goal_timings = pd.read_csv('../data/FS_Champ2021_Goal_timings.csv')

In [68]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [69]:
games = games[0:552]
games
#Dropping playoff matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,over/under
0,1599849900,Sep 11 2020 - 6:45pm,complete,NaN,Watford,Middlesbrough,Keith Stroud,1.0,0.00,0.00,...,3.45,4.05,1.39,2.20,4.15,8.25,1.95,1.74,Vicarage Road (Watford),0
1,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Birmingham City,Brentford,Tony Harrington,1.0,0.00,0.00,...,3.50,1.87,1.33,2.00,3.65,7.00,1.87,1.80,St Andrew's Trillion Trophy Stadium (Birmingham),0
2,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Wycombe Wanderers,Rotherham United,James Linington,1.0,0.00,0.00,...,3.30,2.50,1.31,1.95,3.50,6.70,1.74,1.95,Adams Park (High Wycombe- Buckinghamshire),0
3,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,AFC Bournemouth,Blackburn Rovers,Gavin Ward,1.0,0.00,0.00,...,3.60,4.05,1.28,1.91,3.25,6.25,1.74,1.95,Vitality Stadium (Bournemouth- Dorset),1
4,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Barnsley,Luton Town,Jarred Gillett,1.0,0.00,0.00,...,3.60,4.00,1.30,1.95,3.40,6.60,1.77,1.91,Oakwell (Barnsley- South Yorkshire),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,1620473400,May 08 2021 - 11:30am,complete,NaN,Middlesbrough,Wycombe Wanderers,Martin Atkinson,46.0,1.68,0.64,...,3.70,3.95,1.27,1.82,3.00,5.25,1.74,2.00,Riverside Stadium (Middlesbrough),1
548,1620473400,May 08 2021 - 11:30am,complete,NaN,Nottingham Forest,Preston North End,Darren Bond,46.0,1.18,1.45,...,3.30,3.05,1.32,1.92,3.25,6.00,1.71,2.00,The City Ground (Nottingham- Nottinghamshire),1
549,1620473400,May 08 2021 - 11:30am,complete,NaN,Queens Park Rangers,Luton Town,Tony Harrington,46.0,1.55,1.32,...,3.50,3.10,1.27,1.68,3.00,5.25,1.67,2.10,Kiyan Prince Foundation Stadium (London),1
550,1620473400,May 08 2021 - 11:30am,complete,NaN,Reading,Huddersfield Town,Jarred Gillett,46.0,1.77,0.77,...,3.55,3.30,1.28,1.90,2.80,5.25,1.71,2.00,Madejski Stadium (Reading- Berkshire),1


In [70]:
games['home_xg'] = ""
games['away_xg'] = ""

games['home_xg'] = np.where(games['home_team_name'] == 'Watford', 1.36, games['home_xg'])
games.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,over/under,home_xg,away_xg
0,1599849900,Sep 11 2020 - 6:45pm,complete,NaN,Watford,Middlesbrough,Keith Stroud,1.0,0.0,0.0,...,1.39,2.20,4.15,8.25,1.95,1.74,Vicarage Road (Watford),0,1.36,
1,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Birmingham City,Brentford,Tony Harrington,1.0,0.0,0.0,...,1.33,2.00,3.65,7.00,1.87,1.80,St Andrew's Trillion Trophy Stadium (Birmingham),0,,
2,1599910200,Sep 12 2020 - 11:30am,complete,NaN,Wycombe Wanderers,Rotherham United,James Linington,1.0,0.0,0.0,...,1.31,1.95,3.50,6.70,1.74,1.95,Adams Park (High Wycombe- Buckinghamshire),0,,
3,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,AFC Bournemouth,Blackburn Rovers,Gavin Ward,1.0,0.0,0.0,...,1.28,1.91,3.25,6.25,1.74,1.95,Vitality Stadium (Bournemouth- Dorset),1,,
4,1599919200,Sep 12 2020 - 2:00pm,complete,NaN,Barnsley,Luton Town,Jarred Gillett,1.0,0.0,0.0,...,1.30,1.95,3.40,6.60,1.77,1.91,Oakwell (Barnsley- South Yorkshire),0,,


Let's write a function to help fill xg in our games df

In [71]:
def fill_xg(team, xg):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg, games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg, games['away_xg'])

In [83]:
# team_stats[['common_name', 'xg_for_avg_overall']].sort_values(by='xg_for_avg_overall', ascending=False)

In [75]:
fill_xg('Norwich City', 1.89)
fill_xg('Blackburn Rovers', 1.75)
fill_xg('Brentford', 1.55)
fill_xg('Queens Park Rangers', 1.47)
fill_xg('AFC Bournemouth', 1.44)
fill_xg('Middlesbrough', 1.44)
fill_xg('Barnsley', 1.43)
fill_xg('Millwall', 1.37)
fill_xg('Watford', 1.36)
fill_xg('Reading', 1.36)
fill_xg('Cardiff City', 1.35)
fill_xg('Nottingham Forest', 1.33)
fill_xg('Rotherham United', 1.32)
fill_xg('Wycombe Wanderers', 1.32)
fill_xg('Luton Town', 1.29)
fill_xg('Stoke City', 1.28)
fill_xg('Swansea City', 1.15)
fill_xg('Derby County', 1.11)
fill_xg('Huddersfield Town', 1.11)
fill_xg('Preston North End', 1.11)
fill_xg('Coventry City', 1.11)
fill_xg('Sheffield Wednesday', 0.94)
fill_xg('Birmingham City', 0.94)
fill_xg('Bristol City', 0.81)
#Values based on table above to help fill avg xg for each club

In [85]:
games.columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

# Now let's do some modeling

In [86]:
features = ['home_xg', 'away_xg']
X = games[features]
y = games['over/under']

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)